# Introduction
   A manager at the bank is disturbed with more and more customers leaving their credit card services. They would really appreciate if someone could predict who will churn so they can proactively go to the customer to provide them better services and turn customers' decisions in the opposite direction. 
   
Now, this dataset consists of 10,000 customers mentioning their age, salary, marital_status, credit card limit, credit card category, etc.

In [ ]:
#Make sure you import your dependent packages

from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold

# Step 1
   Step one with any data is to import it. 
    
   Below we have the statement importing the data. Please note that I import two data sets. The first is a data set I have manually numericized the data, and the second is the original data. There is a way to numericize categorical data using sklearn's preprocessing library and the Labelencoder method. But, I have already done my analysis in this format. 
    
   Following the import, we take that DataFrame and we split it into the training data. 
   - X_train is all the lines except the targeted output variable, and the client ID.
   - Y_train is the targeted output variable.        

In [ ]:
df_cc = pd.read_csv(filepath_or_buffer="/kaggle/input/bankchurnersnumeric/BankChurnersNumeric.csv")
y_train = df_cc['Attrition_Flag']
x_train = df_cc.drop(['Attrition_Flag','CLIENTNUM'], axis = 1)

df_cc = pd.read_csv(filepath_or_buffer="/kaggle/input/credit-card-customers/BankChurners.csv")
y_viz = df_cc['Attrition_Flag']
x_viz = df_cc.drop(['Attrition_Flag','CLIENTNUM', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1' ,'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'], axis = 1)

# Step 2
You have to know what you are working with so you must perform some exploratory analysis on the variables.

The first variable you should investigate is the format of the output variable. In our case it is a categorical data point. 1 meaning you are an existing customer and 0 meaning you are a churned customer. 


In [ ]:
print("Sample size of 1= Existing user and 0= Churned user before synthetic oversampling")
print(y_viz.value_counts())

Notice the large discrepency between the amount of data for Existing customers and Churned customers. This difference in data points is what we call bias error. This will lead our model to underfit the data related to the churned customers. In the end this results in the model miss the small relevent relationships in the data. This will lead to predictions that are not accurate towards deciding who is considered a churned customer. In order to remedy this situation we will introduce some synthetically generated data points in order to balance the samples in order to fit the model better.

This will be done using the SMOTE or Synthetic Minority Oversampling Technique. How this technique works in laymans terms is that SMOTE will select similar data points from the data we are trying to over sample. Next a line is drawn between two data points and a point is placed randomly along that line. This will upsample our data will have a similar sample size.

In [ ]:
#Create an oversampled data set to account for the large difference in existing customers vs. attired customers
ov_sample= SMOTE()
x_train,y_train = ov_sample.fit_resample(x_train,y_train)
print("Sample size of 1= Existing user and 0= Churned user post synthetic oversampling")
print(y_train.value_counts())


Notice how the sample sizes are now equal. This is important because if we were to sample more than the amount of existing customers. We may run into a situation where the data is now being overfitted. This will cause the model to be extremely sensitive to the small fluctiations in the data.

The concept of balancing the model's fit is considered the bias-variance tradeoff, and this is an import concept to learn in data science. 
https://en.wikipedia.org/wiki/Bias%E2%80%93variance_tradeoff

# Step 3
   Defining helper functions for the analysis
   
   The first helper function is the cross_validation function. This function serves the purpose of creating testing data. The number of folds determines how many times the model will be trained. How it works is that it divides the data up equally into 10 parts. 9/10 parts become the training data. The remaining part becomes the testing data. This is repeated n-fold times until each subdivision is able to be the testing data for one iteration. 
   One thing to not is that scikit learn has a built in cross_validate method. But I have my own function because I am returning two variable while scikit learn's method only supports one return variable.
   
   The second helper function is a prediciton check. It is designed to take a look at the amount of correct prediction and the amount of acceptable predictions.
   
   - (TP/(TP+FN)) represents the amount of Churned users / (churned users + Existing users who were marked as Churned)
   - (TP+TN+FN)/(TP+TN+FN+FP) represents (Churned users + Existing users + Existing users who were marked as Churned) / (Churned users + Existing users + Existing users who were marked as Churned + Churned users who were marked as Existing)
   
   

In [ ]:
#Helper functions for the data

def cross_validation(algo, X_train, Y_train, folds = 10):
    kf = KFold(n_splits = folds, shuffle=True)
    acc = []
    acc_tot = []
    matrix = None
    first = True

    i=1
    for train_index, test_index in kf.split(X_train, Y_train):
        fx_train, fx_test = X_train.iloc[train_index,:], X_train.iloc[test_index,:]
        fy_train, fy_test = Y_train[train_index],Y_train[test_index]
        algo.fit(fx_train, fy_train)
        fy_pred = algo.predict(fx_test)
        a, b = prediction_check(fy_pred, fy_test)
        acc.append(a)
        acc_tot.append(b)
        i+=1
    acc = pd.Series(a)
    acc_tot = pd.Series(b)
    return acc.mean(), acc_tot.mean()

def prediction_check(y_predict,y_actual):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    #My condition is predicting if a customer will switch from existing(1) to attired(0)
    for i in range(len(y_predict)):
        #Correct prediction of an attired user
        if y_predict[i] == 0 and y_actual.iloc[i] == 0:
            TP+=1
        #Correct prediction of an existing user
        elif y_predict[i] == 1 and y_actual.iloc[i] == 1:
            TN+=1
        #Prediction of an existing user who is actually an attired user
        elif y_predict[i] == 1 and y_actual.iloc[i] == 0:
            FN+=1
        #Prediction of an attired user who is actually an existing user
        elif y_predict[i] == 0 and y_actual.iloc[i] == 1:
            FP+=1
    return (TP/(TP+FN)), (TP+TN+FN)/(TP+TN+FN+FP)




# Step 4
   Deciding on the proper model to use
   
   Given the data, we know we will be using a classification model. I have chosen to compare the results between the Random forest classification modet and the Decition tree classification model. It is always a good idea to test the fit from several models in order to see which works best. 

In [ ]:
#Decision Tree Classifier
print("Running Calculations for DecisionTreeClassifier")
fit = DecisionTreeClassifier(random_state=1).fit(x_train, y_train)
score1, score2 = cross_validation(fit, x_train, y_train)
print("Completed with an TP/(TP+FN) accuracy of: ", score1)
print("Completed with an TP+TN+FN/(TP+TN+FN+FP) accuracy of: ", score2, "\n")

#Random Forest Classifier
print("Running Calculations for RandomForestClassifier")
fit = RandomForestClassifier(random_state=1).fit(x_train, y_train)
score1, score2 = cross_validation(fit, x_train, y_train)
print("Completed with an TP/(TP+FN) accuracy of: ", score1)
print("Completed with an TP+TN+FN/(TP+TN+FN+FP) accuracy of: ", score2)

In this case it is shown that the Random Forest classifier has shown optimal results compared to the Decision tree classifier. 

# Conclusion

   As shown by the TP/(TP+FN) accuracy output above we have a ~99% accuracy in predicting which users will be churned customers out of all users who were marked as churned. The next steps following this analysis will be to apply the trained model to the user base to see if the model marks any existing users as Churned. Following you should monitor and record data of the status of the marked clients without intervention to understand if the model is accurately classifying potentially churned customers. Following a successful monitoring period, intervention techniques should be discussed, and which methods would prove beneficial. Then create sample groups for each type of intervention technique and record data to analyze which intervention technique is the most efective for which groups of individuals for effective targeted intervention in the future. 
   
   Out of all the users who are predicted to be churned. Roughly 1% of them will be classfied as existing users. This could be due to the margin of error because with any predicion. There is no way of being 100% certain an individual will churn or not. Maybe they got offered a better rate at a different bank, and they are loyal customer who happened to make a financially smart decision. I think 1/100 individuals is an acceptable tolerance given the technology availiable to us. If this is applied to J.P. Morgan Chase, which had a total net revunue of 115 billion USD in the year 2019, and claims to serve 63 million households. First, we need to calculate the amount of wrong predictions among the 63 million clients. This comes out to 63,000 clients, let us assume each household is worth 14,000 USD each year. The yearly financial impact of the wrong predictions equate to 882 million USD per year. This is a .007% impact, and can easily be outpaced by projected revenue growth of a company on that scale. According to YoY revenue growth between 2018 and 2019, J.P. Morgan had a .94% increase in revenue which is about 6 billion USD. Given this knowledge our 1% error will only affect YoY revenue growth by .147%. 
   
   A presentation is formed with all the relevant information and explainations about the overall financial impacts and costs associated with the transition to the new model. Then executive leadership would make a decision on whether the provided information justifies the transitsion to the new model, or the executive leadership will request more research be conducted about this aspect of the financial impact. 